<center><img src="https://synapsedatascience.com.br/wp-content/uploads/2020/12/logo-jupyter.png" width="300"/></center>

[TODO]   
1 - Adicionar eventos de outros meses  
2 - Melhorar layout

In [1]:
import pandas as pd
import geopandas as gpd
import folium

from ipywidgets import widgets, Layout, GridBox
from datetime import datetime, time, timedelta, date
from matplotlib import pyplot as plt

# importar o plugin FastMarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
class App():
    
    def __init__(self, events, geometry, lat_id='LATITUDE', lon_id='LONGITUDE', type_col='tipo_evento',
                 date_col='DATAOCORRENCIA', time_col='HORAOCORRENCIA'):
        
        self.events = events
        self.tmp_events = None
        self.geometry = geometry
        
        self.lat_id = lat_id
        self.lon_id = lon_id
        self.type_col = type_col
        
        self.wid_out_map = widgets.Output(
            layout=Layout(width='auto', grid_area='wid_out_map'))
        
        # date picker
        self.wid_datepicker_start = widgets.DatePicker(
            description='Data inicial',
            value = date(2020, 1, 1),
            layout=Layout(width='auto', grid_area='wid_datepicker_start'))
        self.wid_datepicker_end = widgets.DatePicker(
            description='Data final',
            value = date(2020, 12, 31),
            layout=Layout(width='auto', grid_area='wid_datepicker_end'))

        # Time interval widget
        self.wid_selection_inithour = widgets.SelectionSlider(
            options=[str(i)+":00h" for i in range(0, 24)],
            value="0:00h",
            description='Hora inicial',
            layout=Layout(width='auto', grid_area='wid_selection_inithour'))
        self.wid_selection_finalhour = widgets.SelectionSlider(
            options=[str(i)+":59h" for i in range(0, 24)],
            value="23:59h", description='Hora final',
            layout=Layout(width='auto', grid_area='wid_selection_finalhour'))
        
        # Events types
        self.wid_select_notselected = widgets.SelectMultiple(
            disabled=False,
            layout=Layout(width='auto', grid_area='wid_select_notselected'))
        self.wid_select_selected = widgets.SelectMultiple(
            disabled=False,
            layout=Layout(width='auto', grid_area='wid_select_selected'))
        self.wid_button_addtypes = widgets.Button(
            description='',
            disabled=False,
            button_style='success',
            tooltip='Add events types',
            icon='angle-double-right',
            layout=Layout(width='auto', grid_area='wid_button_addtypes'))
        self.wid_button_removetypes = widgets.Button(
            description='',
            disabled=False,
            button_style='danger',
            tooltip='Remove events types',
            icon='angle-double-left',
            layout=Layout(width='auto', grid_area='wid_button_removetypes'))
        
        # Filter events output histogram
        self.wid_out_eventsbar = widgets.Output(
            layout=Layout(width='auto', grid_area='wid_out_eventsbar')
        )
        self.wid_button_filter = widgets.Button(
            description="Select events",
            layout=Layout(width='auto', grid_area='wid_button_filter'))
        
        self.grid_children = [
            self.wid_datepicker_start,
            self.wid_datepicker_end,
            self.wid_selection_inithour,
            self.wid_selection_finalhour,
            self.wid_out_eventsbar,
            self.wid_button_filter,
            self.wid_select_notselected,
            self.wid_select_selected,
            self.wid_button_addtypes,
            self.wid_button_removetypes,
            self.wid_out_map
        ]
        
        self.events['datetime'] = pd.to_datetime(events[date_col] + ' ' + events[time_col])
        
        self.mean_lat = events[lat_id].mean()
        self.mean_lon = events[lon_id].mean()
        
        self.wid_select_notselected.options = list(self.events[self.type_col].unique())
        self.wid_select_selected.options = []
        
        self.wid_button_filter.on_click(self.filter_button_clicked)
        self.wid_button_addtypes.on_click(self.addtypes_button_clicked)
        self.wid_button_removetypes.on_click(self.removetypes_button_clicked)
        
        self.create_map()
        self.build_layout()
        
    def build_layout(self): 
        self.gridbox_filter = \
            GridBox(children=self.grid_children,
                    layout=Layout(
                        width='100%',
                        grid_template_rows='auto',

                        grid_template_columns='23% 23% 5% 23% 23%',
                        grid_gap='5px',
                        grid_template_areas='''
                            "wid_datepicker_start wid_datepicker_start . wid_out_eventsbar wid_out_eventsbar"
                            "wid_datepicker_end wid_datepicker_end . wid_out_eventsbar wid_out_eventsbar"
                            "wid_selection_inithour wid_selection_inithour . wid_out_eventsbar wid_out_eventsbar"
                            "wid_selection_finalhour wid_selection_finalhour . wid_out_eventsbar wid_out_eventsbar"
                            "wid_select_notselected wid_select_notselected wid_button_addtypes wid_select_selected wid_select_selected"
                            "wid_select_notselected wid_select_notselected wid_button_removetypes wid_select_selected wid_select_selected"
                            "wid_button_filter wid_button_filter . . ."
                            "wid_out_map wid_out_map wid_out_map wid_out_map wid_out_map"
                            ''')
                   )
    
    def render(self):
        """The Function to render the screen
        """
        # show empty histogram
        with self.wid_out_eventsbar:
            plt.title("Event histogram")
            plt.show()

        return self.gridbox_filter
    
    # Add events types button
    def addtypes_button_clicked(self, button):
        """Handler function for the click of add type button

        Parameters
        ----------
        button: dict
            The button that has been clicked
        """
        if len(self.wid_select_notselected.value) > 0:
            selected_values = list(self.wid_select_notselected.value)

            self.wid_select_notselected.options = [value for value in self.wid_select_notselected.options 
                                                   if value not in self.wid_select_notselected.value]

            selected_options = selected_values + list(self.wid_select_selected.options)
            self.wid_select_selected.options = selected_options

    # Remove events types button
    def removetypes_button_clicked(self, button):
        """Handler function for the click of remove type button

        Parameters
        ----------
        button: dict
            The button that has been clicked
        """
        if len(self.wid_select_selected.value) > 0:
            selected_values = list(self.wid_select_selected.value)

            self.wid_select_selected.options = [value for value in self.wid_select_selected.options 
                                                if value not in self.wid_select_selected.value]

            selected_options = selected_values + list(self.wid_select_notselected.options)
            self.wid_select_notselected.options = selected_options
            
    def filter_button_clicked(self, button):
        """Handler function for clck on filter button

        Parameters
        ----------
        button: dict
            the clicked button
        """
        
        self.tmp_events = self.events[(self.events.datetime.dt.date >= self.wid_datepicker_start.value) & 
                                      (self.events.datetime.dt.date <= self.wid_datepicker_end.value)].copy()

        # Create the time object
        begin_hour = int(self.wid_selection_inithour.value.split(':')[0])
        end_hour = int(self.wid_selection_finalhour.value.split(':')[0])
        begin_time = time(begin_hour,0,0)
        end_time = time(end_hour,59,59)

        # Filter events by period of the day
        if begin_hour <= end_hour:
            self.tmp_events = self.tmp_events[(self.tmp_events.datetime.dt.time >= begin_time) & 
                                              (self.tmp_events.datetime.dt.time <= end_time)]
        else:
            self.tmp_events = self.tmp_events[(self.tmp_events.datetime.dt.time >= begin_time) |
                                              (self.tmp_events.datetime.dt.time <= end_time)]

        #Filter events by selected types
        if self.type_col:
            types_selected = list(self.wid_select_selected.options)
            self.tmp_events = self.tmp_events[self.tmp_events[self.type_col].isin(types_selected)]
         
        # Take the number of events before the mindist filter
        num_events_time = len(self.tmp_events) 
        # Check if the number of events is above to zero
        if num_events_time > 0:
            # Create the histogram
            df_datetime = self.tmp_events['datetime']
            df_datetime.groupby([df_datetime.dt.year.rename('year'), 
                                 df_datetime.dt.month.rename('month')]).agg({'count'}).plot(kind='bar')
            plt.legend().remove()
            
        else:
            plt.cla()
        
        self.wid_out_eventsbar.clear_output(wait=True)
        with self.wid_out_eventsbar:
            plt.title("Event histogram")
            plt.show()
            
        self.create_map()
        
    
    def create_map(self):
        
        # criar o mapa, trocando o tiles para 'cartodbpositron'
        fmap = folium.Map(location=[self.mean_lat, self.mean_lon], tiles='cartodbpositron')
        
        # criar o GeoJson da nossa geometria para adicionar ao mapa
        limites = folium.features.GeoJson(self.geometry, 
                                          style_function=lambda feature: {
                                              'color': 'black',
                                              'weight' : 2,
                                              'fillOpacity' : 0.0
                                          })
        # adicionar limites no nosso mapa
        fmap.add_child(limites)
        
        if self.tmp_events is not None:
            # criar o marcador de clusterização
            mc = FastMarkerCluster(self.tmp_events[[self.lat_id, self.lon_id]])

            # adicionar marcador no mapa
            fmap.add_child(mc) 

        #self.ipymap.layout.height = '500px'
        self.wid_out_map.clear_output(wait=True)
        with self.wid_out_map:
            display(fmap)
    

In [3]:
filename = 'dados/simplificado_roubos_furtos_capital.json'

gdf_events = gpd.read_file(filename)

filename = 'dados/capital_são_paulo.json'

gdf_geometry = gpd.read_file(filename)

In [4]:
wid_tab = widgets.Tab(layout=Layout(max_width='960px'))
wid_tab.set_title(0, 'Selecionar eventos')

app_screen = App(gdf_events, gdf_geometry)

wid_tab.children = [app_screen.render()]

wid_tab